# Natural Language Processing 2024 – Ex. 2

**Add the names and ID of the submitting students here:**

1. Yaniv gabay


2. Sahar Asher


3. Hadar Video


In this exercise we will perform the task of Sentiment analysis over the IMDB movie review dataset.

The dataset has around 50K movie reviews with each review labeled as "positive" or "negative".

Our goal is given the review we want to classifiy it as positive or negative, this task is also called "Sentiment Analysis"

Below you can find a suggestion of the order things should be implemented, you can follow this or do it your own way.

The exercise has several stages:

1. Downloading and cleaning the data
2. Running some basic analysis
3. Training a Feed Forward network to perform the task using classification
4. Training a Bi-Dir LSTM to perform the task
5. Playing with paramters to see if we get better results

Please sumbit the notebook after it's running stage. Grade will be given for clean code, with comments and explanations

In [3]:
import nltk
import pandas as pd
from IPython.display import display
import re
!pip install contractions
import contractions
import nltk
nltk.download('averaged_perceptron_tagger')


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

nltk.download('stopwords')

from google.colab import files
import io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.6 MB/s eta 0:00:00


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Dictionary to store uploaded files
uploaded_file = files.upload()

Saving IMDB_Dataset.csv to IMDB_Dataset.csv


# Data download and cleaning

1. Download the IMDB dataset.

2. Clean the data:
* Remove URLs, HTML tags and non-alphanumeric characters
* Remove stop-words (use NLTK)
* Lowercase the dataset
* (Optional) Anything else you think can help...

Show one example of a review before and after this cleaning (find a review which has at least one URL/HTML tag/Non-aplhanumeric characters)



In [7]:
# we tried to spell correct the data but the results very not good
# so we decided to not use it example "muslims" into "musea"
digitplaceholder = 'DIGPLACHOLD'
#this function will export the curr data into csv
#so we can obeserve the data more carfully
#if needed
def export_curr_data_to_csv(num_of_question,df):
    try:
         output_file = "curr_data"+num_of_question+".csv"
         df.to_csv(output_file, index=False)
         return
    except Exception as e:
          print("Error loading data: {}".format(e))
          exit(1)

def clean_text(text):
    #1 Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    #2 Remove URLs
    text = re.sub(r'http\S+', '', text)
    #3 Remove non-alphanumeric characters
    text = re.sub(r'\W', ' ', text)
    #4 Convert to lowercase
    text = text.lower()
    #5 Remove stopwords
    stop_words = set(stopwords.words('english'))
    #maybe we need some stopwords, so decrease them from this list if needed
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])
    #6 turn Digits into Token
    text = re.sub(r'\d+', digitplaceholder, text)
    #7 additional steps if needed

    #maybe remove some special chars

    # will need to see how the next step handle this example

    #should we remove all stop words? prob not
    #example not good - turns into "good"
    #so we need to think of a solution, maybe decrease from
    #the stop words list some of the words that are important
    return text
def load_text(filename):
   file_path = filename
   #IMDB data set has header at 0 row
   data = pd.read_csv(io.BytesIO(uploaded_file['IMDB_Dataset.csv']), header=0, names=['review', 'sentiment'])
   display(data.head())
    # Inspect the data

    # Check for missing values
   #print(data.isnull().sum())
   #print(data.info())

    # Apply the cleaning function to the review column
   data['review'] = data['review'].apply(clean_text)
   return data

# Example usage
example_review = "This is an example review with HTML <b>bold</b> tags and a URL: https://example.com"
cleaned_review = clean_text(example_review)
print("Original:", example_review)
print("Cleaned:", cleaned_review)
try:
# Load the data
    data = load_text('IMDB_Dataset.csv')
    data_orig_copy = data.copy()
    print("cleaned text printing:\n")
    display(data.head())
    print(data.sample(10)['review'])
    export_curr_data_to_csv("1",data)
   # describe = data.describe()
   # print(describe)


except Exception as e:
    print("Error loading data: {}".format(e))
    exit(1)


Original: This is an example review with HTML <b>bold</b> tags and a URL: https://example.com
Cleaned: example review html bold tags url


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


cleaned text printing:



,review,sentiment
0,one reviewers mentioned watching DIGPLACHOLD o...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter mattei love time money visually stunnin...,positive


20041    splicing searchers one weirdest films ever see...
4057     heck good time viewing picture splendidly surp...
2828     things come rarity rarities film ideas many fi...
5132     misfit recruit private owens tests drill instr...
44372    first DIGPLACHOLD minutes movie quite boring f...
14284    honestly best reality show anyone ever come or...
48831    well typical straight toilet slasher film long...
3037     technically abominable audible pops scenes awe...
8684     film viewed right frame mind first central fat...
15193    first impresson saikano live action movie trai...
Name: review, dtype: object


# Tokenization

1. Tokenize the dataset (you can tokenize using spaces or use more robust methods from NLTK)
2. (Optional) Lemmatize the text (you can use NLTK) this can improve results
3. Lemmatize should be carfully be done, so we wont lose too much.
4. Show an example of 3 sentences before and after this process

In [9]:
from nltk.corpus import wordnet
from nltk.tag import pos_tag


#we decided to use POS tagging
# this prevent the lemmatize process from taking
# words like was and convert them into "wa"

#will make she's into she is
def expand_text_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

#need to hansle situations like she's where currently it goes into she 's . so the best solution
#currently is to delete 's from the text
auxiliary_verbs = {"am", "is", "are", "was", "were", "be", "being", "been", "will", "shall", "would", "should", "can", "could", "may", "might", "must", "do", "does", "did", "have", "has", "had"}

def get_wordnet_pos(treebank_tag):
    """Map POS tag to the format accepted by WordNetLemmatizer."""
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(treebank_tag[0].upper(), wordnet.NOUN)
def tokenize_and_lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    #print("POS Tags:", pos_tags)
    lemmatized_tokens = []
    for token, pos in pos_tags:
        if token in auxiliary_verbs:
            # Skip lemmatization for auxiliary verbs
            lemmatized_tokens.append(token)
        else:

            lemma = lemmatizer.lemmatize(token, get_wordnet_pos(pos))
            lemmatized_tokens.append(lemma)

    return ' '.join(lemmatized_tokens)



# Example sentences
example_sentences = [
    "The cats are chasing the mice.",
    "He was running late for the meeting.",
    "She's not enjoying the sunny weather."
]


# Process and display the examples
for sentence in example_sentences:
    expanded_sentence = expand_text_contractions(sentence)
    processed = tokenize_and_lemmatize(expanded_sentence)
    print("Original:", sentence)
    print("Processed:", processed, "\n")

Original: The cats are chasing the mice.
Processed: The cat are chase the mouse . 

Original: He was running late for the meeting.
Processed: He was run late for the meeting . 

Original: She's not enjoying the sunny weather.
Processed: She is not enjoy the sunny weather . 



# Basic analysis

Perfrom some analysis on the data
1. Show the number percentage of negative/positive review (label balancing)
2. Plot some statistics on the length of review (after our cleaning process)
3. (Optional) show anything else you think is important

In [10]:
#beware long run time if not on google colab
# 6-7 min run time (POS tagging takes time)
def final_preprocess(review):
    # First, expand contractions
    expanded_review = expand_text_contractions(review)
    # Then, tokenize and lemmatize the expanded review
    processed_review = tokenize_and_lemmatize(expanded_review)
    return processed_review

# Apply the preprocess_review function to each review in the DataFrame
#we created another col here, instead of rewriting the data.
#we wanted to maybe play with both results, and see how good
#tokenize_and_lemmatize and expand_text_contractions are.
data['processed_review'] = data['review'].apply(final_preprocess)

#data right now is :
#1.review |2. sentiment |3. processed_review
#:
#1.review - is after preprocessing
#2.sentiment - is the original data (havent changed)
#3.processed_review - is after clean and final preprocess so proccesed.

# Display the first few rows to verify the changes
print(data[['review', 'processed_review']].head())




                                              review  \
0  one reviewers mentioned watching DIGPLACHOLD o...   
1  wonderful little production filming technique ...   
2  thought wonderful way spend time hot summer we...   
3  basically family little boy jake thinks zombie...   
4  petter mattei love time money visually stunnin...   

                                    processed_review  
0  one reviewer mention watch DIGPLACHOLD oz epis...  
1  wonderful little production film technique una...  
2  think wonderful way spend time hot summer week...  
3  basically family little boy jake think zombie ...  
4  petter mattei love time money visually stunnin...  


In [11]:
##### label balancing
label_counts = data_orig_copy['sentiment'].value_counts(normalize=True) * 100
print(label_counts)



positive    50.0
negative    50.0
Name: sentiment, dtype: float64


In [12]:
# Calculating the length of each cleaned review
data['review_length'] = data['processed_review'].apply(len)

# Displaying basic statistics
print("Statistics on the length of cleaned reviews:")
print(data['review_length'].describe())


Statistics on the length of cleaned reviews:
count    50000.000000
mean       794.851240
std        613.754239
min         17.000000
25%        416.000000
50%        584.500000
75%        966.000000
max       8832.000000
Name: review_length, dtype: float64


# Preparing the dataset for training
we can also use glove or previously used models as the first layer
1. Choose your vocabulary size K (should be between 1000 and 3000)
2. Find the top K frequent words in your database
3. Create word indexes like we did in class, for any word not in your top K  words replace with \<UNK\>. Remember to add an index for the \<PAD\> token.
4. Create a new dataset with indexes instead of words later to be used for training
5. Convert your labels to numeric representation (that your network can deal with).

Split the dataset to 80% traind and 20% test, remember to keep the balance between labels!
we need to make sure, we still have enough labels on both sides



In [ ]:
# Your code here

# Training a feed forward neural network

For simplicity we would take only reviews with 500 words (after tokenization) or less.
For this part we would train a neural network that gets the full review as one input (like we had in our NER example in class) and outputs the label (positive or negative).
Remember that you need to PAD the words so all reviews will have the same length.

For this section please try at least 3 variants of different network and show if the results change, you can choose from the following:
1. Adding hidden layers to the network
2. Running with and without Dropout
3. Trying different optimizers

(Optional) Try to use the Glove embedding: Create an embedding layer in your PyTorch model using the loaded GloVe embeddings. You will initialize the weights of the embedding layer with the GloVe embeddings.

For each option:

* Plot the train and test error during training, does your network overfit?

* Plot the final results of the network, including accuracy and confusion matrix

In [ ]:
# Your code here

# Training a BiDir LSTM neural network

Now do the same as the prvious section with a bi-directional LSTM.

Remember that the output of the LSTM should be connected to a small feed forward network to perform the actual classification.

Here again you can play with number of layers and the LSTM or the small network of the output. Show only the best result you got.

* Plot the train and test error during training, does your network overfit?

* Plot the final results of the network, including accuracy and confusion matrix

Are the results better than the previous section?




Finally show 3 reviews from the test data with correct labales and 3 without, why do you think the network did not success on these examples?